In [11]:
from credential import client_id, client_secret, username, password, user_agent
import praw
import csv


In [ ]:

reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     username=username,
                     password=password,
                    user_agent= user_agent)

In [12]:
def get_post_urls(subreddit_name, limit=100):
    subreddit = reddit.subreddit(subreddit_name)
    post_urls = [submission.url for submission in subreddit.new(limit=limit)]
    return post_urls

subreddit_name = 'VietNam'
post_urls = get_post_urls(subreddit_name, limit=300)

# Print the URLs
for i, url in enumerate(post_urls):
    print(f"Post {i + 1}: {url}")

Post 1: https://www.reddit.com/gallery/1cvr9r0
Post 2: https://www.reddit.com/r/VietNam/comments/1cvqtov/where_to_see_lotus_flowers_in_bloom_near_da_nang/
Post 3: https://www.reddit.com/r/VietNam/comments/1cvq8b1/high_grade_flowers_in_saigon/
Post 4: https://www.reddit.com/r/VietNam/comments/1cvpwl1/silver_chain_cleaning_in_hcm/
Post 5: https://www.reddit.com/r/VietNam/comments/1cvpeoy/ha_giang_loop_experienced_rider_after_proper/
Post 6: https://www.reddit.com/r/VietNam/comments/1cvowk3/tips/
Post 7: https://i.redd.it/4guqyhof1e1d1.jpeg
Post 8: https://www.reddit.com/r/VietNam/comments/1cvolm9/anyone_have_any_ideas_where_a_16_years_old_can/
Post 9: https://www.reddit.com/r/VietNam/comments/1cvoh0x/where_to_find_3d_scanningphotogrammetry_vendors/
Post 10: https://www.reddit.com/r/VietNam/comments/1cvodi9/before_i_decide_i_want_your_sage_advice/
Post 11: https://www.reddit.com/r/VietNam/comments/1cvn6yq/httpstuoitrenewsvnnewssociety20240519foreignersfil/
Post 12: https://www.reddit.com/

In [13]:
from datetime import datetime, timedelta

def get_weekday_binary_features(date):
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    weekday = date.strftime("%A")
    return {f"Basetime Weekday {i+1}": int(weekday == day) for i, day in enumerate(weekdays)}

def get_post_weekday_binary_features(date):
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    weekday = date.strftime("%A")
    return {f"Post Weekday {i+1}": int(weekday == day) for i, day in enumerate(weekdays)}

def crawl_reddit_comments(post_url):
    # Fetch the post by URL
    submission = reddit.submission(url=post_url)
    submission.comments.replace_more(limit=None)  # Load all comments

    # Current time as basetime
    basetime_dt = datetime.now()

    # Time calculations
    T1 = basetime_dt - timedelta(hours=48)
    T2 = basetime_dt - timedelta(hours=24)
    post_time = datetime.fromtimestamp(submission.created_utc)
    time_diff_hours = (basetime_dt - post_time).total_seconds() / 3600  # Time difference in hours

    # Filter and count comments based on time criteria
    total_comments_before_basetime = sum(1 for c in submission.comments.list() if datetime.fromtimestamp(c.created_utc) < basetime_dt)
    comments_last_24h = sum(1 for c in submission.comments.list() if T2 <= datetime.fromtimestamp(c.created_utc) < basetime_dt)
    comments_T1_T2 = sum(1 for c in submission.comments.list() if T1 <= datetime.fromtimestamp(c.created_utc) < T2)
    comments_first_24h_after_post = sum(1 for c in submission.comments.list() if post_time <= datetime.fromtimestamp(c.created_utc) < post_time + timedelta(hours=24))

    # Length of the post
    post_length = len(submission.selftext)

    # Binary features for weekdays
    basetime_binary_features = get_weekday_binary_features(basetime_dt)
    post_binary_features = get_post_weekday_binary_features(post_time)

    return {
        "text": submission.selftext,
        "Total Comments Before Basetime": total_comments_before_basetime,
        "Comments in Last 24 Hours Before Basetime": comments_last_24h,
        "Comments Between T1 and T2": comments_T1_T2,
        "Comments in First 24 Hours After Post": comments_first_24h_after_post,
        "Different between attr2 and attr3": comments_last_24h - comments_T1_T2,
        "Time Difference (hours)": time_diff_hours,
        "Post Length": post_length,
        **basetime_binary_features,
        **post_binary_features
    }

In [14]:
def crawl_multiple_posts(post_urls):
    results = []
    for url in post_urls:
        try:
            result = crawl_reddit_comments(url)
            
            results.append(result)
        except Exception as e:
            print(f"Error crawling {url}: {e}")
    return results

def save_to_csv(data, filename='reddit_comments.csv'):
    keys = data[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)


In [15]:
comment_data = crawl_multiple_posts(post_urls)
if comment_data:  # Check if there is any data to save
    save_to_csv(comment_data)
    print("Data saved to CSV file.")
else:
    print("No data to save.")

Error crawling https://i.redd.it/4guqyhof1e1d1.jpeg: Invalid URL: https://i.redd.it/4guqyhof1e1d1.jpeg
Error crawling https://i.redd.it/5pbfw7e14c1d1.jpeg: Invalid URL: https://i.redd.it/5pbfw7e14c1d1.jpeg
Error crawling https://i.redd.it/6xxq9c3d2c1d1.jpeg: Invalid URL: https://i.redd.it/6xxq9c3d2c1d1.jpeg
Error crawling https://i.redd.it/z8rrakqwqb1d1.jpeg: Invalid URL: https://i.redd.it/z8rrakqwqb1d1.jpeg
Error crawling https://v.redd.it/wye2gc9vla1d1: received 404 HTTP response
Error crawling https://www.reuters.com/world/asia-pacific/vietnam-nominates-top-police-official-lam-state-president-2024-05-18/: Invalid URL: https://www.reuters.com/world/asia-pacific/vietnam-nominates-top-police-official-lam-state-president-2024-05-18/
Error crawling https://i.redd.it/ol6f73kq781d1.jpeg: Invalid URL: https://i.redd.it/ol6f73kq781d1.jpeg
Error crawling https://i.redd.it/hw5ohkwe771d1.png: Invalid URL: https://i.redd.it/hw5ohkwe771d1.png
Error crawling https://i.redd.it/a863mjfkk61d1.jpeg: I

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Load the dataset
file_path = 'reddit_comments.csv'
df = pd.read_csv(file_path)

# Extract text data
texts = df['text'].fillna('')

df=df.drop('text',axis=1)
# Initialize the CountVectorizer to get the 200 most frequent words
vectorizer = CountVectorizer(max_features=200, binary=True)
X = vectorizer.fit_transform(texts)

# Get the feature names (the 200 most frequent words)
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame with the binary features
binary_features_df = pd.DataFrame(X.toarray(), columns=feature_names)

# Concatenate the binary features with the original DataFrame
# Find the insertion point (after 'Post Length' column)
insertion_index = df.columns.get_loc('Post Length') + 1

# Split the original DataFrame into two parts and insert the new features
df_part1 = df.iloc[:, :insertion_index]
df_part2 = df.iloc[:, insertion_index:]

# Combine the parts with the new binary features in between
updated_df = pd.concat([df_part1, binary_features_df, df_part2], axis=1)

# Save the updated dataset to a new CSV file
output_file_path = 'reddit_comments_with_features.csv'
updated_df.to_csv(output_file_path, index=False,header=False)

print(f"Updated dataset saved to {output_file_path}")


Updated dataset saved to reddit_comments_with_features.csv


In [22]:
file_path = 'reddit_comments_with_features.csv'
df = pd.read_csv(file_path,header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,211,212,213,214,215,216,217,218,219,220
0,1,1,0,1,1,0.512728,529,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0.837827,101,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,1.276261,55,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,1.523023,86,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,3,3,0,3,3,1.920905,936,0,1,1,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,21,0,0,18,0,158.689598,320,0,0,0,...,0,0,1,1,0,0,0,0,0,0
247,4,0,0,4,0,159.368882,459,0,0,0,...,0,0,1,1,0,0,0,0,0,0
248,7,0,0,7,0,160.787325,599,0,0,1,...,0,0,1,1,0,0,0,0,0,0
249,21,0,0,19,0,161.970792,154,0,0,0,...,0,0,1,1,0,0,0,0,0,0
